In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch03SparkToolSet-Streaming3").getOrCreate()

In [3]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

u'http://gw02.itversity.com:4043'

In [4]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

http://rm01.itversity.com:19288/proxy/application_1533622723243_12037
rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1533622723243_12037
http://gw02.itversity.com:4043


In [5]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv")

In [7]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [8]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [12]:
staticDataFrame.show(truncate=False, n=5)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|580538   |23084    |RABBIT NIGHT LIGHT             |48      |2011-12-05 08:38:00|1.79     |14075.0   |United Kingdom|
|580538   |23077    |DOUGHNUT LIP GLOSS             |20      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22906    |12 MESSAGE CARDS WITH ENVELOPES|24      |2011-12-05 08:38:00|1.65     |14075.0   |United Kingdom|
|580538   |21914    |BLUE HARMONICA IN BOX          |24      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22467    |GUMBALL COAT RACK              |6       |2011-12-05 08:38:00|2.55     |14075.0   |United Kingdom|
+---------+---------+---------------------------

In [13]:
sc.version

u'2.3.0.2.6.5.0-292'

In [16]:
# COMMAND ----------
from pyspark.sql.functions import window, column, desc, col
t1 = staticDataFrame\
  .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
  .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [17]:
 t1.show(n=5, truncate=False)

+----------+------------------------------------------+-----------------+
|CustomerId|window                                    |sum(total_cost)  |
+----------+------------------------------------------+-----------------+
|16057.0   |[2011-12-04 19:00:00, 2011-12-05 19:00:00]|-37.6            |
|14126.0   |[2011-11-28 19:00:00, 2011-11-29 19:00:00]|643.6300000000001|
|13500.0   |[2011-11-15 19:00:00, 2011-11-16 19:00:00]|497.9700000000001|
|17160.0   |[2011-11-07 19:00:00, 2011-11-08 19:00:00]|516.8499999999999|
|15608.0   |[2011-11-10 19:00:00, 2011-11-11 19:00:00]|122.4            |
+----------+------------------------------------------+-----------------+
only showing top 5 rows



In [31]:
staticDataFrame\
  .selectExpr("CustomerId", "UnitPrice", "Quantity", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
  .where(col("CustomerId") == '16057.0').where(col("InvoiceDate") >= '2011-12-04 19:00:00')\
.where(col("InvoiceDate") <= '2011-12-05 19:00:00').show()

+----------+---------+--------+----------+-------------------+
|CustomerId|UnitPrice|Quantity|total_cost|        InvoiceDate|
+----------+---------+--------+----------+-------------------+
|   16057.0|    12.75|      -1|    -12.75|2011-12-05 16:36:00|
|   16057.0|     9.95|      -2|     -19.9|2011-12-05 16:36:00|
|   16057.0|     4.95|      -1|     -4.95|2011-12-05 16:36:00|
+----------+---------+--------+----------+-------------------+



In [18]:
t1.take(1)

[Row(CustomerId=16813.0, window=Row(start=datetime.datetime(2011, 11, 30, 19, 0), end=datetime.datetime(2011, 12, 1, 19, 0)), sum(total_cost)=148.54000000000002)]

In [19]:
t1.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[CustomerId#16, window#132], functions=[sum(total_cost#122)])
+- Exchange hashpartitioning(CustomerId#16, window#132, 200)
   +- *(1) HashAggregate(keys=[CustomerId#16, window#132], functions=[partial_sum(total_cost#122)])
      +- *(1) Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) as double) = (cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) THEN (CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) + 1) ELSE CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) END + 0) - 1) * 86400000000) + 0), LongType, TimestampType), end, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, Long

In [20]:
t1.printSchema()

root
 |-- CustomerId: double (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- sum(total_cost): double (nullable = true)



In [32]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [33]:
# COMMAND ----------
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv")

In [34]:
streamingDataFrame.explain() # note the Error

== Physical Plan ==
org.apache.spark.sql.AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
FileSource[/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv]


In [35]:
# COMMAND ----------
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [36]:
# COMMAND ----------
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [37]:
# COMMAND ----------

spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(20)

+----------+------+---------------+
|CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



In [39]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `window` DESC
  """)\
  .show(20)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   15605.0|[2010-11-30 19:00...|194.85000000000002|
|   14142.0|[2010-11-30 19:00...|311.81000000000006|
|   17968.0|[2010-11-30 19:00...|277.34999999999997|
|   17069.0|[2010-11-30 19:00...|            277.05|
|   15100.0|[2010-11-30 19:00...|             350.4|
|   15525.0|[2010-11-30 19:00...| 313.9299999999999|
|   17643.0|[2010-11-30 19:00...|101.55000000000001|
|   16928.0|[2010-11-30 19:00...|            116.25|
|   16552.0|[2010-11-30 19:00...|             95.29|
|   12838.0|[2010-11-30 19:00...|390.78999999999985|
|   13767.0|[2010-11-30 19:00...| 507.8800000000001|
|   16456.0|[2010-11-30 19:00...|             787.4|
|   16098.0|[2010-11-30 19:00...|430.59999999999997|
|   15235.0|[2010-11-30 19:00...|              79.5|
|   14307.0|[2010-11-30 19:00...| 783.1099999999999|
|   14606.0|[2010-11-30 19:00...|            1

In [ ]:
# purchaseByCustomerPerHour.writeStream\
#     .format("console")\
#     .queryName("customer_purchases_2")\
#     .outputMode("complete")\
#     .start()

#check on Console